# Canny Benchmark

## TODO:

* Canny-end-to-end
* Memoization
* Overlay
* Benchmark notebook

## Imports & Setup

In [2]:
%load_ext autoreload
%autoreload 2

import sys
import os

sys.path.append("../")

enable_cudasim = False
if enable_cudasim:
    # Enable the CUDA simulator
    os.environ["NUMBA_OPT"] = "0"
    os.environ["NUMBA_ENABLE_CUDASIM"] = "1"
    os.environ["NUMBA_CUDA_DEBUGINFO"] = "1"

import logging
import time
import pathlib
from pathlib import Path
import importlib.util
from importlib import reload
import math
from io import BytesIO
import unittest.mock as mock
import gc

import matplotlib as mpl
import matplotlib.pyplot as plt

from icecream import ic

import numpy as np
import pandas as pd
import scipy as sp
import numba
from numba import cuda

import cv2

from IPython.display import clear_output, display, Image
import ipywidgets as widgets
from PIL import Image as PILImage

from utils.benchmarking import time_function, time_line
from utils.setup_notebook import init_notebook, source_code_path_is_from_notebook
from utils.setup_logging import setup_logging
from utils.plotting_tools import (
    SmartFigure,
    to_ipy_image,
    plot_image,
    plot_kernel,
    plot_matrix,
)

init_notebook()
setup_logging("INFO")


## Loading Canny Implementations

In [3]:
# Load canny implementations
dir_canny_impls = "./canny_impls"
files_canny_impls = [f for f in os.listdir(dir_canny_impls) if str(f).endswith(".py")]
files_canny_impls.sort()

if "canny_impls" not in locals():
    ic("CANNYIMPLARRAY_RESET!")
    canny_impls = []


def load_module(file_path: str, dyn_modules: list):
    module_name = Path(file_path).stem
    module_string = f"{module_name}"

    already_loaded = module_string in sys.modules
    if already_loaded:
        with time_line(f"Reloading {module_name}"):
            module = sys.modules[module_string]
            reload(module)
        return module

    with time_line(f"Loading {module_name}"):
        folder = str(Path(file_path).parent)
        if folder not in sys.path:
            sys.path.append(folder)
        spec = importlib.util.spec_from_file_location(module_string, file_path)
        module = importlib.util.module_from_spec(spec)
        sys.modules[module_string] = module
        spec.loader.exec_module(module)

    dyn_modules.append(module)

    return module


with time_line(f"Loading {len(files_canny_impls)} canny implementations"):
    for f in files_canny_impls:
        full_path = Path(dir_canny_impls) / f
        module = load_module(full_path, canny_impls)


2024-10-23 06:51:24.087 INFO root Loading 2 canny implementations started (C:\Users\_\AppData\Local\Temp\ipykernel_8556\401445511.py:36)
2024-10-23 06:51:24.105 INFO root Loading rd_numba_cuda_fp32 started (C:\Users\_\AppData\Local\Temp\ipykernel_8556\401445511.py:22)
2024-10-23 06:51:24.127 INFO numba.cuda.cudadrv.driver init


ic| 'CANNYIMPLARRAY_RESET!'


2024-10-23 06:51:26.948 INFO root Loading rd_numba_cuda_fp32 took: 2.8419 s (C:\Users\_\AppData\Local\Temp\ipykernel_8556\401445511.py:22)
2024-10-23 06:51:26.987 INFO root Loading rd_vec_v4_dibit started (C:\Users\_\AppData\Local\Temp\ipykernel_8556\401445511.py:22)
2024-10-23 06:51:27.567 INFO root Loading rd_vec_v4_dibit took: 580.4565 ms (C:\Users\_\AppData\Local\Temp\ipykernel_8556\401445511.py:22)
2024-10-23 06:51:27.568 INFO root Loading 2 canny implementations took: 3.4808 s (C:\Users\_\AppData\Local\Temp\ipykernel_8556\401445511.py:36)


## Loading Input Images

In [4]:
image_input_dir = "./image_input"
input_files = [
    f for f in os.listdir(image_input_dir) if f.endswith(".jpg") or f.endswith(".png")
]
input_files.sort()

images_color = []
images_gray = []

with time_line(f"Loading {len(input_files)} images"):
    for f in input_files:
        with time_line(f"Loading {f}"):
            file_image = Path(image_input_dir) / f
            image_color = cv2.imread(str(file_image), cv2.IMREAD_COLOR)
            images_color.append(image_color)
            # image_gray = (
            #    cv2.cvtColor(image_color, cv2.COLOR_BGR2GRAY).astype(np.float32) / 255.0
            # )
            image_gray = cv2.cvtColor(image_color, cv2.COLOR_BGR2GRAY)
            images_gray.append(image_gray)

# sort all three lists by image size
images_color, images_gray, input_files = zip(
    *sorted(zip(images_color, images_gray, input_files), key=lambda x: x[0].size)
)

2024-10-23 06:51:27.957 INFO root Loading 5 images started (C:\Users\_\AppData\Local\Temp\ipykernel_8556\2264552153.py:10)
2024-10-23 06:51:27.980 INFO root Loading circle_128.png started (C:\Users\_\AppData\Local\Temp\ipykernel_8556\2264552153.py:12)
2024-10-23 06:51:27.981 INFO root Loading circle_128.png took: 762.9000 µs (C:\Users\_\AppData\Local\Temp\ipykernel_8556\2264552153.py:12)
2024-10-23 06:51:28.006 INFO root Loading circle_32.png started (C:\Users\_\AppData\Local\Temp\ipykernel_8556\2264552153.py:12)
2024-10-23 06:51:28.007 INFO root Loading circle_32.png took: 452.4000 µs (C:\Users\_\AppData\Local\Temp\ipykernel_8556\2264552153.py:12)
2024-10-23 06:51:28.028 INFO root Loading circle_64.png started (C:\Users\_\AppData\Local\Temp\ipykernel_8556\2264552153.py:12)
2024-10-23 06:51:28.029 INFO root Loading circle_64.png took: 362.6000 µs (C:\Users\_\AppData\Local\Temp\ipykernel_8556\2264552153.py:12)
2024-10-23 06:51:28.046 INFO root Loading park.png started (C:\Users\_\AppDat

## Running Canny

In [5]:
# Show a dropdown to select the image

output = widgets.Output()

image_dropdown = widgets.Dropdown(
    options=input_files,
    description="Image",
    disabled=False,
)
canny_impl_dropdown = widgets.Dropdown(
    options=[canny_impl.__name__ for canny_impl in canny_impls],
    description="Canny Implementation",
    disabled=False,
    value=canny_impls[0].__name__,
)
sigma_slider = widgets.FloatSlider(
    value=3.0,
    min=0.1,
    max=10.0,
    step=0.1,
    description="Sigma",
    disabled=False,
    continuous_update=False,
    orientation="horizontal",
    readout=True,
    readout_format=".1f",
)

fig_width = 16
image_size = 512

plot_gauss_smart_fig = SmartFigure()
plot_non_max_smart_fig = SmartFigure()
plot_hysteresis_smart_fig = SmartFigure()


@output.capture(clear_output=True, wait=True)
def on_menu_change(change=None):
    # reload the impl module
    current_canny_impl = canny_impl_dropdown.value
    load_module(Path(dir_canny_impls) / f"{current_canny_impl}.py", canny_impls)

    # Show the selected image
    image_color = images_color[image_dropdown.index]
    image_gray = images_gray[image_dropdown.index]

    height, width = image_gray.shape

    logging.info(f"Selected image: {image_dropdown.value} ({width}x{height})")
    logging.info(f"Selected canny implementation: {canny_impl_dropdown.value}")
    with time_line("Displaying input images"):
        display(
            widgets.HBox(
                [
                    to_ipy_image(image_color, longest_side=image_size, upscale=True),
                    to_ipy_image(image_gray, longest_side=image_size, upscale=True),
                ]
            )
        )

    canny_impl = next(
        canny_impl
        for canny_impl in canny_impls
        if canny_impl.__name__ == canny_impl_dropdown.value
    )

    # GAUSS
    with time_line("Blurring image"):
        image_blurred = canny_impl.blur_gauss(image_gray, sigma_slider.value)
    with time_line("Displaying blurred image"):
        display(to_ipy_image(image_blurred, longest_side=image_size, upscale=True))

    plot_gauss = False
    if plot_gauss:
        global plot_gauss_smart_fig
        plot_gauss_smart_fig = SmartFigure(
            figsize=(
                fig_width,
                fig_width / 2,
            ),  # rc_params={"figure.constrained_layout.use": True}
        )
        fig_gauss = plot_gauss_smart_fig.get_fig()
        ax = fig_gauss.add_subplot(1, 2, 1), fig_gauss.add_subplot(1, 2, 2)
        for ax_ in ax:
            if hasattr(ax_, "clear"):
                ax_.clear()
        plot_matrix(ax[0], image_gray, title="Original Image")
        plot_matrix(ax[1], image_blurred, title="Blurred Image")

        # ax_3d = fig.add_subplot(2, 2, 3, projection="3d")
        # kernel = image_blurred[:19, :19]
        # plot_kernel(ax_3d, kernel, title="Kernel")

        fig_gauss.tight_layout()
        fig_gauss.canvas.layout.min_width = "400px"
        fig_gauss.canvas.layout.flex = "1 1 auto"
        fig_gauss.canvas.layout.width = "auto"
        # fig.canvas.resizable = False
        display(fig_gauss.canvas)

    # SOBEL
    with time_line("Calculating sobel gradients"):
        grad_mag, grad_dir = canny_impl.sobel_gradients(image_blurred)

    grad_mag = grad_mag.astype(np.float32)
    grad_dir = grad_dir.astype(np.float32)

    grad_dir_color = cv2.applyColorMap(
        np.uint8((grad_dir + np.pi) / (2 * np.pi) * 255),
        cv2.COLORMAP_RAINBOW,
    )
    grad_dir_color = grad_dir_color.astype(np.float32) / 255.0
    image_gradients = np.append(
        cv2.cvtColor(grad_mag, cv2.COLOR_GRAY2BGR), grad_dir_color, axis=1
    )
    display(to_ipy_image(image_gradients, longest_side=image_size, upscale=True))

    # NON-MAXIMUM SUPPRESSION
    with time_line("Non-maximum suppression"):
        image_nms = canny_impl.non_max(grad_mag, grad_dir)

    plot_non_max = False
    if plot_non_max:
        global plot_non_max_smart_fig
        plot_non_max_smart_fig = SmartFigure(
            figsize=(
                fig_width,
                fig_width / 2,
            ),  # rc_params={"figure.constrained_layout.use": True}
        )
        fig_non_max = plot_non_max_smart_fig.get_fig()
        ax = fig_non_max.add_subplot(1, 2, 1), fig_non_max.add_subplot(1, 2, 2)
        for ax_ in ax:
            if hasattr(ax_, "clear"):
                ax_.clear()
        plot_matrix(ax[0], grad_mag, title="Gradient Magnitude")
        plot_matrix(ax[1], image_nms, title="Non-Maximum Suppression")

        fig_non_max.tight_layout()
        fig_non_max.canvas.layout.min_width = "400px"
        fig_non_max.canvas.layout.flex = "1 1 auto"
        fig_non_max.canvas.layout.width = "auto"
        # fig.canvas.resizable = False
        display(fig_non_max.canvas)
    # with time_line("Displaying non-maximum suppression"):
    #    display(to_ipy_image(image_nms, longest_side=image_size, upscale=True))

    # HYSTERESIS AUTO THRESHOLDING
    do_auto_thresholding = True
    if do_auto_thresholding:
        with time_line("Hysteresis auto-thresholding"):
            low_high_prop = np.array([0.7, 0.3], dtype=np.float32)
            low_high = canny_impl.compute_hysteresis_auto_thresholds(
                image_nms, low_high_prop
            )
            low, high = low_high[0], low_high[1]
        logging.info(f"Low threshold: {low}, High threshold: {high}")
        ic("Low threshold", low, "High threshold", high)
    else:
        low = 0.7
        high = 0.3
        low_high = np.array([low, high], dtype=np.float32)

    # HYSTERESIS

    debug_hyst_cuda_shared_mem_copy = False
    if debug_hyst_cuda_shared_mem_copy and image_nms.shape[0] == image_nms.shape[1]:
        # DEBUG CUDA
        image_nms[image_nms <= np.finfo(np.float32).eps] = 0.1

        # top left
        image_nms[0, 0] = 0.6
        image_nms[2, 0] = 0.4
        image_nms[0, 2] = 0.5
        # bottom left
        image_nms[-1, 0] = 0.6
        image_nms[-3, 0] = 0.4
        image_nms[-1, 2] = 0.5
        # top right
        image_nms[0, -1] = 0.6
        image_nms[2, -1] = 0.4
        image_nms[0, -3] = 0.5
        # bottom right
        image_nms[-1, -1] = 0.6
        image_nms[-3, -1] = 0.4
        image_nms[-1, -3] = 0.5

        middle = image_nms.shape[0] // 2
        # middle middle
        image_nms[middle, middle] = 0.6
        image_nms[middle + 2, middle] = 0.4
        image_nms[middle, middle + 2] = 0.5
        image_nms[middle - 2, middle] = 0.4
        image_nms[middle, middle - 2] = 0.5
        image_nms[middle + 1, middle + 1] = 0.3
        image_nms[middle - 1, middle - 1] = 0.3
        image_nms[middle + 1, middle - 1] = 0.3
        image_nms[middle - 1, middle + 1] = 0.3
        image_nms[middle + 1, middle] = 0.2
        image_nms[middle - 1, middle] = 0.2
        image_nms[middle, middle + 1] = 0.2
        image_nms[middle, middle - 1] = 0.2
        # top middle
        image_nms[0, middle] = 0.6
        image_nms[2, middle] = 0.4
        image_nms[0, middle + 2] = 0.5
        image_nms[0, middle - 2] = 0.5
        # bottom middle
        image_nms[-1, middle] = 0.6
        image_nms[-3, middle] = 0.4
        image_nms[-1, middle + 2] = 0.5
        image_nms[-1, middle - 2] = 0.5
        # left middle
        image_nms[middle, 0] = 0.6
        image_nms[middle, 2] = 0.5
        image_nms[middle + 2, 0] = 0.4
        image_nms[middle - 2, 0] = 0.4
        # right middle
        image_nms[middle, -1] = 0.6
        image_nms[middle, -3] = 0.5
        image_nms[middle + 2, -1] = 0.4
        image_nms[middle - 2, -1] = 0.4

        image_nms[image_nms == 1.0] = 0.9
        image_nms[28, 16] = 1.0
        image_nms[30, 30] = 0.8

    with time_line("Hysteresis"):
        image_edges = canny_impl.hysteresis(image_nms, low_high)

    plot_hysteresis = True
    if plot_hysteresis:
        global plot_hysteresis_smart_fig
        plot_hysteresis_smart_fig = SmartFigure(
            figsize=(
                fig_width,
                fig_width / 2,
            ),  # rc_params={"figure.constrained_layout.use": True}
        )
        fig_hysteresis = plot_hysteresis_smart_fig.get_fig()
        ax = fig_hysteresis.add_subplot(1, 2, 1), fig_hysteresis.add_subplot(1, 2, 2)
        for ax_ in ax:
            if hasattr(ax_, "clear"):
                ax_.clear()
        plot_matrix(ax[0], image_nms, title="Non-Maximum Suppression")
        plot_matrix(ax[1], image_edges, title="Edges")

        fig_hysteresis.tight_layout()
        fig_hysteresis.canvas.layout.min_width = "400px"
        fig_hysteresis.canvas.layout.flex = "1 1 auto"
        fig_hysteresis.canvas.layout.width = "auto"
        # fig.canvas.resizable = False
        display(fig_hysteresis.canvas)
    with time_line("Displaying edges"):
        display(to_ipy_image(image_edges, longest_side=image_size, upscale=True))


image_dropdown.observe(on_menu_change, names="value")
canny_impl_dropdown.observe(on_menu_change, names="value")
sigma_slider.observe(on_menu_change, names="value")

display(
    widgets.VBox(
        [widgets.HBox([image_dropdown, canny_impl_dropdown, sigma_slider]), output]
    )
)
on_menu_change()